<a href="https://colab.research.google.com/github/samyzaf/notebooks/blob/main/heat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2D Heat Equation Simulator using FDM
* **FDM** stands for **Finite Difference Method**.
* This is a method for solving differential equations
  numerically, which is explained in details at the course
  lecture notes (add reference later?)
* Additional explanations are provided in the project
  booklet.
* Heat equation simulator Python code was inspired by
  G. Nervadof at his medium blog:  
  https://levelup.gitconnected.com/solving-2d-heat-equation-numerically-using-python-3334004aa01a

* You need to install our **fdmtools Python Package**.
* Please run the next code cell to install and export these tools.

In [1]:
%pip install -q https://samyzaf.com/fdmtools-1.zip
from fdmtools import *

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


* We only touch one example of the heat equation over a
  rectangular plate (in fact a square) which has
  the following general form

* Additional similar problems can be solved in the same manner.
  
<IMG src="https://samyzaf.com/fdm/heat2b.jpg" width=800 align="center"/>

## Basic parameters
As mentioned, we will use a square plate (**L=a=b**)
with the following FDM parameters
* **L** = Plate Length = Width = Height
* **N** = Grid division (dx = dy = L/N)
* **alpha** = Heat parameter
* **dt** = Time grid unit
* **fps** = Frames Per Second (for video)
* **seconds** = Video Simulation time (also video duration)
* **num_frames** = Total number of **time frames** in our simulation
* **dx = dy =** Plate Length grid units (L/N)
* **frames** = Video frames range (integer list from 0 to num_frames-1)
* **pbar** = Progress Bar object for monitoring the progress of creating our video simulation.
* **U** = 3D Numpy Array

## Our test case example: simple square plate `LxL`, `L=4`
* To make it as simple as possible we will work on
  a square domain with length **`L=a=b=4`**.

* Exact coordinates: **`D = [0,4]x[0,4]`**

* We will use the **Finite Difference Method (FDM)**
  to solve the following problem on a 4x4 plate  
    
$\qquad
\left\{
\begin{array}{ll}
u_t = 0.1(u_{xx} + u_{yy}), &\quad 0<x<4,\quad 0<y<4,\quad 0\leq t<\infty
\\
u(x,y,0) = 0,  &\quad 0<x<4,\quad 0<y<4,\quad t=0
\\
u(x,0,t) = 100\,\sqrt[3]{x/4},  &\quad 0\leq x\leq 4,\quad 0\leq t<\infty
\\
u(x,4,t) = 100(0.7 + 0.3\sin\frac{5\pi x}{4}),  &\quad 0\leq x\leq 4,\quad 0\leq t<\infty
\\
u(0,y,t) = 100\sqrt{y/4},  &\quad 0\leq y\leq 4,\quad 0\leq t<\infty
\\
u(4,y,t) = 0,  &\quad 0\leq y\leq 4,\quad 0\leq t<\infty
\end{array}
\right.
$

* Here is our specific FDM parameter plan for this problem:

In [3]:
L = 4.0
N = 50
alpha = 0.1
fps = 30      # frames per second (for video)
seconds = 10  # video duration
num_frames = seconds*fps
dx = dy = L/N
#dt = 0.016                   #Not recommended, better use an automatic choice
dt = dx**2 / (4 * alpha)      # A better automatic choice for dt!
print(f"Good choice for dt = {dt}")
gamma = (alpha*dt) / (dx**2)
frames = range(0, num_frames)
pbar = ProgressBar(num_frames, prompt="Animating: ")

U = np.empty((N, N, num_frames))   # 3D Numpy Array - this is our FDM grid!

Good choice for dt = 0.016


## 3D Grid Array
* In the last line, we created an initially empty
  3D [Numpy](https://numpy.org)
  Array **`U`**,
  which is the container of our 3D Grid model for
  simulating the heat equation.

* The shape of **`U`** is: **`N x N x num_frames`**.

* Use **`U[i,j,k]`** to access pixel **`(i,j)`**
  on the plate at time frame **`k`**.

* [Numpy](https://numpy.org/) is a the standard scientific
  computation Python package, which is popular for numerical
  computations in many engineering fields.


<IMG src="https://samyzaf.com/fdm/cover3.jpg" width=800 align="center"/>

## Solver
* The **solve** method solves the heat equation for a given
  3d array **U**.
* This method simulates the temperature frames on
  **all plate points** for the full time duration,
  and saves them in the array **U**.
* For every pixel **`(i,j)`**, and every time frame **`k`**,
  **`U[i,j,k]`** is the temperature of the plate at pixel
  **`(i,j)`**, at time frame **`k`**.
* We use the **[Numba package](https://numba.pydata.org)
  just in time compiler** for speed enhacement
  (thanks to Intel/Nvidia support).
* It translates the Python code to C and
  gets about 1000x speed enhancement.

In [4]:
@njit
def solve(U):
  for k in range(0, num_frames-1):
    for i in range(1, N-1):
      for j in range(1, N-1):
        S = U[i+1,j,k] + U[i-1,j,k] + U[i,j+1,k] + U[i,j-1,k] - 4*U[i,j,k]
        U[i,j,k+1] = U[i,j,k] + gamma * S
  return U

## Initial/Boundary conditions
* Before solving our grid, we need to set initial
  and boundary conditions for it.
* We start with the initial
  temperature `f(x,y)` on the whole plate at
  time frame t=0.
* Then we set the boundary conditions on the plate sides
  using 4 functions `f1, f2, g1, g2`.
* See details in the following figure.

<IMG src="https://samyzaf.com/fdm/heat5e.jpg" width=700 align="center"/>



In [5]:

f = lambda i,j: 0
f1 = lambda j: 100 * (j*dx/L) ** (1/3)
f2 = lambda i: 100 * (0.7 + 0.3*sin(5*pi*i*dx/L))
g1 = lambda j: 100 * (j*dy/L)**(1/2)
g2 = lambda i: 0

U[:,:,0] = [[f(i,j) for i in range(N)] for j in range(N)]

# Bottom side (row=0,column=all,time=all)
U[0,:,:] = [[f1(i)] for i in range(N)]

# Top side (row=N-1, column=all, time=all)
U[N-1,:,:] =  [[f2(i)] for i in range(N)]

# Left side (row=all,column=0,time=all)
U[:,0,:] = [[g1(j)] for j in range(N)]

# Right side (row=all, column=N-1, time=all)
U[:,N-1,:] = [[g2(j)] for j in range(N)]

## Comments
* The Python **lambda** command is a simple mechanism
  for creating one line functions.
* The FDM mapping from grid indices **U[i,j,k]**
  to **x,y,t** values
  is as follows:
  * **$\mathbf{x_i}$ = i*dx/N**
  * **$\mathbf{y_j}$ = j*dy/N**
  * **$\mathbf{t_k}$ = k*dt**
  * **U[i,j,k] = $\mathbf{u(x_i,y_j,t_k)}$**
* The **":"** symbol in Python stands for the
  default full range **"0:N"**.
* So, for example, the expression **`U[:,:,0]`** is
  actually **`U[0:N, 0:N, 0]`**
  which is the matrix for the temperature
  at time frame **`k=0`**.
* The expression **`U[0,:,:]`** is actually
  **`U[0, 0:N, 0:num_frames]`** which represents the
  matrix for the bottom side of the plate over all time frames.

## Temperature Frame Plotter
* We need a method for plotting the plate heat state
  at time frame **`k`**.

* We have copied the following two methods from
  [G. Nervadof's blog](https://levelup.gitconnected.com/solving-2d-heat-equation-numerically-using-python-3334004aa01a),
  and added two lines for tick labels formatting.

* The argument **`U_k`** is the **`k`**-th slice of **`U`**
  (the **`k`**-th temperature frame).

* **plt** is the matplotlib canvas
  * `clf` - Clears the current plot figure
  * `gca` - Get current axis

In [6]:
def plot_heatmap(U_k, k):
    plt.clf()
    plt.title(f"Temperature at time t = {k*dt:.3f}")
    plt.xlabel("x")
    plt.ylabel("y")
    ax = plt.gca()
    ax.xaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"{x/N*L:0.1f}"))
    ax.yaxis.set_major_formatter(FuncFormatter(lambda y, pos: f"{y/N*L:0.1f}"))

    # This is to plot U_k (U at time-step k)
    plt.pcolormesh(U_k, cmap=plt.cm.jet, vmin=0, vmax=100)
    plt.colorbar()
    return plt

def animate(k):
    plot_heatmap(U[:,:,k], k)
    pbar.advance()


* Now we are ready to solve our equation.
* The **solve** method accepts the empty array **U** and
  fills it with the correct grid values.
* We use the [matplotlib animation package](https://matplotlib.org/stable/users/explain/animations/animations.html)
  for generating an **mp4** video from the sequence of
  temperature frames.
* Video is saved in file, and then played using the
  **play_video** command.

In [7]:
U = solve(U)

anim = animation.FuncAnimation(plt.figure(), animate, interval=1, frames=frames, repeat=False)
video_file = "test1.mp4"
anim.save(video_file, fps=fps)
print("Done!")

print("Playing video file")
play_video(video_file, width=640)

Animating: 100%   
Time: 120.86 seconds
Done!
Playing video file


## Comments
* Note that *video run time* is
  not idetical to real *physical simulation run time*!
* The physical heat phenomenon may take a very short time,
  but the video must slow it down so we can observe it slowly.
* So in general, video duration is much
  larger than physical duration.
* Video run time is defined by the **seconds** parameter.
  Physical run time can be calculated by multiplying
  the number of time frames by **dt**:

In [ ]:
print(f"Video run time = {seconds} seconds")
print(f"Simulation run time = {num_frames*dt} seconds")

* To generate higher resolution videos, you may need
  to increase the size of the parameters
  **`N`**, and **seconds**.
* You may try **`N=50`** or **`N=100`**, but it will
  take much more time to generate the video.
* You may also want to increase the video time.
  You may try **`seconds=30`** or **`seconds=60`**.

* However, when you make such changes, the old value
  of **`dt`** may not work and it is hard to guess
  a good value.
* Fortunately there is a formula
  for guessing a good value for **`dt`** which we used in
  our code.
* For convenience, we have gathered all the needed code
  in **one code cell**, which you can run
  **repeatedly** without restarting
  the notebook. Use it for experimenting with other cases.


In [11]:
L = 4.0
N = 100
alpha = 0.1
fps = 30 # fps = frames per second
seconds = 40
num_frames = seconds*fps
dx = dy = L/N
dt = dx**2 / (4 * alpha)      # A good choice for dt!
print(f"Good choice for dt = {dt}")
gamma = (alpha*dt) / (dx**2)
frames = range(0, num_frames)
pbar = ProgressBar(num_frames, prompt="Animating: ")

U = np.empty((N, N, num_frames))
f = lambda i,j: 0
f1 = lambda j: 100 * (j*dx/L) ** (1/3)
f2 = lambda i: 100 * (0.7 + 0.3*sin(5*pi*i*dx/L))
g1 = lambda j: 100 * (j*dy/L)**(1/2)
g2 = lambda i: 0

U[:,:,0] = [[f(i,j) for i in range(N)] for j in range(N)]
U[0,:,:] = [[f1(i)] for i in range(N)]
U[N-1,:,:] =  [[f2(i)] for i in range(N)]
U[:,0,:] = [[g1(j)] for j in range(N)]
U[:,N-1,:] = [[g2(j)] for j in range(N)]

@njit
def solve(U):
  for k in range(0, num_frames-1):
    for i in range(1, N-1):
      for j in range(1, N-1):
        S = U[i+1,j,k] + U[i-1,j,k] + U[i,j+1,k] + U[i,j-1,k] - 4*U[i,j,k]
        U[i,j,k+1] = U[i,j,k] + gamma * S
  return U

U = solve(U)

anim = animation.FuncAnimation(plt.figure(), animate, interval=1, frames=frames, repeat=False)
video_file = "test2.mp4"
anim.save(video_file, fps=fps)
print("Done!")

play_video(video_file, width=640)

Good choice for dt = 0.004
Animating: 100%   
Time: 401.29 seconds
Done!


* You can download your video file to your local
  computer by running the following command.

In [ ]:
#file_download("test2.mp4")

* You can also plot particular heat maps at a particular
  time frame using the following code.

In [ ]:
#plot_heatmap(U[:,:,18], 18)
#plt.show()